<a href="https://colab.research.google.com/github/ryuaus26/Google-Colab/blob/main/FashionMnist%20with%20Custom%20Functionalities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import keras
import matplotlib.pyplot
import numpy as np
from tqdm import tqdm
from keras.utils import plot_model

In [ ]:
from keras.layers.core.dense import Dense
class CustomDenseLayer(keras.layers.Layer):
  def __init__(self,units,activation= None):
    super().__init__()
    self.units = units
    self.activation = activation

  def build(self,input_shape):
    w_init = tf.random_normal_initializer()
    self.w = tf.Variable(initial_value=w_init(shape=(input_shape[-1],self.units),dtype='float32'),trainable=True)
    b_init = tf.zeros_initializer()
    self.b = tf.Variable(initial_value=b_init(shape=(self.units,),dtype='float32'),trainable=True)

  def call(self,inputs):
    if self.activation == tf.nn.softmax or self.activation=="softmax":
      output = tf.nn.softmax(tf.matmul(inputs, self.w) + self.b,axis=-1)
    elif self.activation is not None:
      output = tf.matmul(inputs, self.w) + self.b
    else:
      output = self.activation(tf.matmul(inputs, self.w) + self.b)
    return output


def base_model():
  inputs = keras.layers.Input(shape=(784,))
  x = CustomDenseLayer(units=64,activation='relu')(inputs)
  x = CustomDenseLayer(units=64,activation='relu')(x)
  outputs = CustomDenseLayer(units=10,activation='softmax')(x)
  model= keras.Model(inputs=inputs,outputs=outputs)
  return model

train_data = tfds.load('fashion_mnist',split='train')
test_data = tfds.load('fashion_mnist',split='test')

def format_image(data):
  image = data['image']
  # Flattened to a 1D vector of length 784
  image = tf.reshape(image,[-1])
  image = tf.cast(image,tf.float32)
  image = image/255.0
  return image, data['label']

# the keyword map() applies the function format_image across all the images stored in train_data and test_data variables
train_data = train_data.map(format_image)
test_data = test_data.map(format_image)

#Allows for faster data
batch_size = 64
train = train_data.shuffle(buffer_size=1024).batch(batch_size)
test = test_data.batch(batch_size=batch_size)

In [ ]:
loss_object = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.Adam()
train_metrics = tf.metrics.CategoricalAccuracy()
val_metrics = tf.metrics.CategoricalAccuracy()


In [ ]:
model = base_model()
plot_model(model, to_file='custom_model.png', show_shapes=True, show_layer_names=True)

def apply_gradient(model,x,y,optimizer):
  with tf.GradientTape() as tape:
    logits = model(x)
    loss_value = loss_object(y_true=y,y_pred=logits)

  gradients = tape.gradient(loss_value,model.trainable_weights)
  optimizer.apply_gradients(zip(gradients,model.trainable_weights))


  return logits,loss_value


def train_one_epoch():
  pbar = tqdm(total=len(list(enumerate(train))), position=0, leave=True, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} ')
  losses = []
  for step, (x_train_batch,y_train_batch) in enumerate(train):
    logits,loss_value = apply_gradient(model,x_train_batch,y_train_batch,optimizer)
    train_metrics.update_state(y_train_batch,logits)
    losses.append(loss_value)
    pbar.set_description(f"Training loss for step {step} {loss_value}")
    pbar.update()
  return losses

def perform_validation():
  losses = []
  for (x_val,y_val) in test:
    logits = model(x_val)
    val_loss = loss_object(logits,y_val)
    val_metrics.update_state(y_val,logits)
    losses.append(val_loss)
  return losses
model.summary()

Model: "model_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_44 (InputLayer)       [(None, 784)]             0         
                                                                 
 custom_dense_layer_104 (Cus  (None, 64)               50240     
 tomDenseLayer)                                                  
                                                                 
 custom_dense_layer_105 (Cus  (None, 64)               4160      
 tomDenseLayer)                                                  
                                                                 
 custom_dense_layer_106 (Cus  (None, 10)               650       
 tomDenseLayer)                                                  
                                                                 
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
______________________________________________________

In [ ]:
#Start training

EPOCHS = 20
epoch_val_losses, epoch_train_losses = [], []

for epoch in (range(EPOCHS)):
  print("Start of epoch :{}".format(epoch))

  losses_train = train_one_epoch()
  train_acc = train_metrics.result()

  losses_val = perform_validation()
  val_acc = val_metrics.result()

  losses_train_mean = np.mean(losses_train)
  epoch_train_losses.append(losses_train_mean)
  losses_val_mean = np.mean(losses_val)
  epoch_val_losses.append(losses_val_mean)
  print("Epoch: {} Train Loss: {}  Val Loss: {}  Train Accuracy: {} Val Accuracy {} ".format(epoch,losses_train,losses_val,train_acc,val_acc))
  train_acc.reset()
  val_acc.reset()



Start of epoch :0


Training loss for step 230 0.530408501625061:  25%|██▍       | 231/938 